## Get Static Fight Stats

In [30]:
# Import required libraries
from bs4 import BeautifulSoup
import requests
import string
import csv

# Initialize the CSV file for writing
with open('ufc_fighters_static.csv', 'w', newline='') as csvfile:
    # Define the column headers for the CSV
    fieldnames = ['Name', 'Height', 'Reach', 'Stance', 'DOB', 'ID']
    
    # Initialize the CSV writer
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write the header row to the CSV
    writer.writeheader()

    # Generate a list of lowercase alphabets to iterate through
    alphabets = list(string.ascii_lowercase)

    # Loop through each alphabet letter
    for letter in alphabets:
        # Fetch the fighter list page for the current alphabet letter
        source = requests.get(f'http://www.ufcstats.com/statistics/fighters?char={letter}&page=all').text
        soup = BeautifulSoup(source, "lxml")

        # Loop through each fighter entry on the page
        for fighter in soup.find_all('tr', attrs={'class': 'b-statistics__table-row'}):
            a_href = fighter.find('a')
            
            # Check if the fighter entry has a URL
            if a_href is not None:
                # Fetch the fighter's individual stats page
                source2 = requests.get(a_href.get("href")).text
                
                # Extract the fighter ID from the URL
                ID = a_href.get("href").split('/')[-1].strip()
                
                soup2 = BeautifulSoup(source2, "lxml")

                # Extract and store the fighter's name
                name = soup2.find('span', {'class': 'b-content__title-highlight'}).text.strip()

                # Initialize a dictionary to hold the fighter's stats
                fighter_stats = {'Name': name, 'ID': ID}

                # Extract and store other stats like Height, Reach, Stance, and DOB
                stats = soup2.find_all('li', attrs={'class': 'b-list__box-list-item'})
                for stat in stats:
                    cleaned_stat = [string.strip() for string in stat.text.split(":")]
                    if len(cleaned_stat) > 1:
                        key = cleaned_stat[0]
                        value = cleaned_stat[1]
                        fighter_stats[key] = value

                # Write the collected stats to the CSV file
                writer.writerow(fighter_stats)
